In [0]:
from collections import defaultdict
from fractions import Fraction
import numpy as np

## **Arithmatic Encoder:**

In [0]:
# make frequency table ( calculate the probabilty of each symbol that repreasents the width ,and the probabilty that represents the start of each symbol range)
# inputs: the input sequence
# output: 1.prob_table....> it is a dictionary contains the probability of each symbol onle
#         2.prob_pairs_dic....> it is a dictionary contains the cumulitive probability of each symbol(the start of its range) and the probability of it.
#       * the two outputs are used in the code
# how it works:  it counts the no. of occurence of each symbol using "defultdict" and divide it by the length of the sequence.   
def calculate_prob(input_symbols):
  counts = defaultdict(int)
  for symbol in input_symbols:
        counts[symbol] += 1
               
  prob_table = dict()
  length = len(input_symbols)
  total_count = 0
  prob_pairs_dic = dict()
  for s in sorted(counts, key=counts.get, reverse=True):
        current_count = counts[s]
        prob_pairs_dic[s] =  Fraction(total_count, length), Fraction(current_count, length)         # dict contain the two values(start, width)    
        prob_table[s] = Fraction(current_count, length)                                             # dict contains width only
        total_count += current_count

  return prob_table , prob_pairs_dic




In [0]:
# encode the symbol range into bits
# input: 1)lb: lower bound of the scale and it sets to "0" at the beginning.
#        2)ub: upper bound of the scale  and it sets to "1" at the beginning.
#        3)curr_lb: lower bound of the symbol range.
#        4)curr_ub: upper bound of the symbol range.
#        5)value: 0 or 1 bit according to the range of the symbol.
# output: code: the encoded bits of the input.
# how it works: it search for the section where the symbol exsist using its range and repeat the process till reach the specific range.
def get_binary(lb,ub,curr_lb,curr_ub,value):
    
    mid = (lb+ub)/2
    if((curr_lb <= lb) and  (curr_ub >= ub)):
        
        return value
                    
        exit()
    else:
        if((curr_lb > mid) and (curr_ub < ub)):
            value.append(1)                 # the symbol range over the mid 
            return get_binary(mid,ub,curr_lb,curr_ub,value)
            

        elif ((curr_ub < mid) and (curr_lb > lb)  ): 
            value.append(0)                           # the symbol range below the mid
            return get_binary(lb, mid, curr_lb, curr_ub, value)
            

        elif((curr_lb > lb ) and (curr_ub > mid)):          # the symbol upper range over thr mid and the symbol lower ranger under the mid
            value.append(1)
            return get_binary(mid,ub, curr_lb, curr_ub,value )
           
   
        else: 
            value.append(0)                                                            # the symbol upper range > upper range  or symbol lower range < lower range
            return get_binary(lb,mid,curr_lb,curr_ub, value)
            

    
    

In [0]:
# get the decimal range of the sequence
#input: 1)lb: lower bound of the scale and it sets to "0" at the beginning.
#       2)ub: upper bound of the scale and it sets to "1" at the beginning. 
#       3)index: the index of the symbols in the sequence and it sets to "0" at the beginning.
#       4)prob_table: probability of each symbol.
#       5) seq:  the input sequence
#       6) x: encoded bits
# how it works: it searches for the range of the symbol in the prob_table and repeat the process till the last symbol then pass the final ranges to "get_binary" function to encode it.
def get_range(lb,ub,index,prob_table,seq):
    if (index == len(seq)):
        Interval = [lb,ub]
        print("Interval =[" + str(lb) + "," + str(ub) + "]")        # Printing the Interval
        curr_lb = lb
        curr_ub = ub
        value = []
        x = get_binary(0, 1,curr_lb,curr_ub,value)    # function call for encoding the interval
        return x      
                             
    else:
      dataSymbol = seq[index]
      lenRange =ub-lb
    
      for sym in prob_table:
        if(sym==dataSymbol):
            
            return get_range(lb,lb+(prob_table.get(sym)*lenRange),index+1,prob_table,seq)
        else:
            lb = lb+(prob_table.get(sym) *lenRange)

    

In [0]:
# main function for encoder:
# input: input sequence
# output:1)prob_pairs: table of probabilities that contains the start and the width of each symbol. (it will be passed to the decoder)
#        2)coded_out: encoded bits
# how it works: it calls the functions " calculate_prob" and "get_range"
def Arithm_encoder(seq):
  
  seq = np.append(seq,[256]) 
  # make the probability table for this sequence
  prob_table, prob_pairs = calculate_prob(seq)
  
  # Apply the arithmatic alogrithm
  coded_out = get_range(0,1,0,prob_table,seq)


  return prob_pairs, coded_out



In [0]:
# main to test the encoder function

sequence = np.array([1,2,1,2,3,5,8,7,5,4])
p,c = Arithm_encoder(sequence)           # data type of p....> dic    and data type of c.....> list

print(p)        
print(c)

Interval =[9927605404/285311670611,902509588/25937424601]
{1: (Fraction(0, 1), Fraction(2, 11)), 2: (Fraction(2, 11), Fraction(2, 11)), 5: (Fraction(4, 11), Fraction(2, 11)), 3: (Fraction(6, 11), Fraction(1, 11)), 8: (Fraction(7, 11), Fraction(1, 11)), 7: (Fraction(8, 11), Fraction(1, 11)), 4: (Fraction(9, 11), Fraction(1, 11)), 256: (Fraction(10, 11), Fraction(1, 11))}
[0, 0, 0, 0, 1, 0, 0, 0, 1, 1, 1, 0, 1, 0, 0, 0, 0, 1, 0, 1, 1, 1, 1, 0, 0, 0, 1, 0, 1, 0, 0, 1, 0, 0]


# **Arithmatic Decoder :**

In [0]:
# convert the binary code into decimal fraction                                         side note for me:( index return the first occurance only)
# input: encoded bits ( the output of the encoder)     
# output: the float number that represents the input bits
# how it works: it does the converstion using the equation:  d = 1*2^(-1) + 0*2^(-2) + .......       for code = 10....           
def binary_to_decimal(code):
  decimal = 0
  for i, c in enumerate(code):
    decimal += c*(2**(-(i+1)))
    

  return decimal 


In [0]:
  # get the original sequence.
  #input:1)decimal: it is the float number that represent the coded bits.
  #      2)prob_ranges: it is the probability table that passed from the encoder.
  # how it works: it take the decimal number and search for its range in the table and emits the symboles while searching till reach the EOF symbol.  ( 256 in this code ) 
  def decode_decimal(decimal, prob_ranges):
    output = []
    code = 257

    while code != 256:
        for code, (start, width) in prob_ranges.items():
            if 0 <= (decimal - start) < width:
                decimal = (decimal - start) / width

                if code < 256:
                    output.append(code)
                break

    return output

In [0]:
# main function for decoder.
# input:1) code: the encoded bits from the encoder.
#       2) probs: it is the probability table that passed from the encoder.
# output: original_s : it is the original sequence.
# how it works: it calls the functions: "binary_to_decimal" & "decode_decimal"
def Arithm_decoder(code,probs):

  float_no = binary_to_decimal(code)
  original_s = decode_decimal(float_no,probs)
  return original_s


In [0]:
# main to test  decoder.
code = c     # from encoder
prob = p     # from encoder
out = Arithm_decoder(c,p)
print(out)

[1, 2, 1, 2, 3, 5, 8, 7, 5, 4]
